# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint
### Not for Grading

## Learning Objective

At the end of the experiment, you will be able to :

* perform Data Pre-processing
* perform Bagging classifier

## Dataset

### Description

The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of many passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

Build a predictive model that answers the question: “what sort of people were more likely to survive?” using titanic's passenger data (ie name, age, gender, socio-economic class, etc).

<br/>

### Data Set Characteristics:

**PassengerId:** Id of the Passenger

**Ticket_Class:** Socio-economic status (SES)
  * 1st = Upper
  * 2nd = Middle
  * 3rd = Lower

**Name:** Surname, First Names of the Passenger

**Sex:** Gender of the Passenger

**Age:** Age of the Passenger

**Siblings_Spouse:**	No. of siblings/spouse of the passenger aboard the Titanic	

**Parents_Children:**	No. of parents / children of the passenger aboard the Titanic	

**Ticket_Number:**	Ticket number

**Fare:** Passenger fare

**Cabin:**	Cabin number

**Embarked:** Port of Embarkation

**Survived:** Survived or Not information

## Setup Steps

In [35]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2100121" #@param {type:"string"}


In [36]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "5142192291" #@param {type:"string"}


In [37]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()
  
notebook= "U1W5_17_Bagging_Classifier" #name of the notebook
Answer = "Ungraded"
def setup():
    from IPython.display import HTML, display
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Titanic.csv")
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "feedback_experiments_input" : Comments, "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aiml.iiith.talentsprint.com/notebook_submissions")
        # print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError 
    else: 
      return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    
else:
  print ("Please complete Id and Password cells before running setup")


Setup completed successfully


In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

## Data Pre-Processing

#### Exercise 01: Load the data and print the first five records

**Hint:** https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html

In [162]:
# YOUR CODE HERE
df = pd.read_csv('Titanic.csv')
print(df.head())

   PassengerId Ticket_Class  ...     Embarked Survived
0            1    3rd class  ...  Southampton       No
1            2    1st class  ...    Cherbourg      Yes
2            3    3rd class  ...  Southampton      Yes
3            4    1st class  ...  Southampton      Yes
4            5    3rd class  ...  Southampton       No

[5 rows x 12 columns]


#### Exercise 02: Data Cleaning

* Generate [Descriptive Statistics](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) of the dataframe

* Count [NaN values in each column](https://stackoverflow.com/a/26266451) of the dataframe

* Fill the blanks in the age column as follows:
  * Fill the age of the survived people with the average age of the survived people
  * Similarly, fill the remaining blanks with the average age of not survived people

  **Hint:** [DataFrame.where](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.where.html) for replacing values with the **`False`** condition

* Drop unnecessary columns which are not contributing for the prediction of a person survival

* Make sure the final dataframe does not have any null or nan values. Delete the rows which has missing values.

*  **Example:**
  * PassengerId column can never decide survival of a person, hence it can be dropped






In [73]:
# YOUR CODE HERE
print(df.describe())
print('\n using info \n')
print(df.info())
print(df.columns)

       PassengerId         Age  Siblings_Spouse  Parents_Children        Fare
count   891.000000  714.000000       891.000000        891.000000  891.000000
mean    446.000000   29.699118         0.523008          0.381594   32.204208
std     257.353842   14.526497         1.102743          0.806057   49.693429
min       1.000000    0.420000         0.000000          0.000000    0.000000
25%     223.500000   20.125000         0.000000          0.000000    7.910400
50%     446.000000   28.000000         0.000000          0.000000   14.454200
75%     668.500000   38.000000         1.000000          0.000000   31.000000
max     891.000000   80.000000         8.000000          6.000000  512.329200

 using info 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PassengerId       891 non-null    int64  
 1   Ticket_Class      891 non-null  

In [74]:
print(df.isnull().sum())
print('\n is na \n')
print(df.isna().sum())
print('\n is is null precesntatiage \n')
print(df.isnull().sum()/len(df) * 100)

PassengerId           0
Ticket_Class          0
Name                  0
Sex                   0
Age                 177
Siblings_Spouse       0
Parents_Children      0
Ticket_Number         0
Fare                  0
Cabin               687
Embarked              0
Survived              0
dtype: int64

 is na 

PassengerId           0
Ticket_Class          0
Name                  0
Sex                   0
Age                 177
Siblings_Spouse       0
Parents_Children      0
Ticket_Number         0
Fare                  0
Cabin               687
Embarked              0
Survived              0
dtype: int64

 is is null precesntatiage 

PassengerId          0.000000
Ticket_Class         0.000000
Name                 0.000000
Sex                  0.000000
Age                 19.865320
Siblings_Spouse      0.000000
Parents_Children     0.000000
Ticket_Number        0.000000
Fare                 0.000000
Cabin               77.104377
Embarked             0.000000
Survived             0.00000

In [148]:
df1 = df.columns
print(list(df1))
print(df1.shape)

['PassengerId', 'Ticket_Class', 'Name', 'Sex', 'Age', 'Siblings_Spouse', 'Parents_Children', 'Ticket_Number', 'Fare', 'Cabin', 'Embarked', 'Survived']
(12,)


In [75]:
 #df.Age = df.Age.fillna(df.Age.mean())
 #print(df.Age)
 Survived_No, Survived_Yes = df.groupby(['Survived'])['Age'].mean()
 print(Survived_No)
 print(Survived_Yes )

30.62617924528302
28.343689655172415


In [154]:
df.columns
df['Survived'] = le.fit_transform(df['Survived'])

In [155]:
 df['Age'] = np.where(df['Survived'] == 'Yes',Survived_Yes, Survived_No)
 print(df.isna().sum())

PassengerId           0
Ticket_Class          0
Name                  0
Sex                   0
Age                   0
Siblings_Spouse       0
Parents_Children      0
Ticket_Number         0
Fare                  0
Cabin               687
Embarked              0
Survived              0
dtype: int64


In [157]:
 df.head()
df['Survived'] = le.fit_transform(df['Survived'])

In [163]:
del df['Cabin']


In [81]:
df.columns

Index(['PassengerId', 'Ticket_Class', 'Name', 'Sex', 'Age', 'Siblings_Spouse',
       'Parents_Children', 'Ticket_Number', 'Fare', 'Cabin', 'Embarked',
       'Survived'],
      dtype='object')

#### Exercise 03: Convert categorical values to numerical
**Hint:** Use [Sklearn LabelEncoder's](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) fit_transform method

In [95]:
# YOUR CODE HERE
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
print(df.columns)

Index(['PassengerId', 'Ticket_Class', 'Name', 'Sex', 'Age', 'Siblings_Spouse',
       'Parents_Children', 'Ticket_Number', 'Fare', 'Cabin', 'Embarked',
       'Survived'],
      dtype='object')


In [119]:
df.Cabin.drop

<bound method Series.drop of 0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object>

In [132]:
df['Sex'] = le.fit_transform(df['Sex'])
df['Name'] = le.fit_transform(df['Name'])
df['Survived'] = le.fit_transform(df['Survived'])
df['Embarked'] = le.fit_transform(df['Embarked'])
df['Ticket_Class'] = le.fit_transform(df['Ticket_Class'])
df['Ticket_Number'] = le.fit_transform(df['Ticket_Number'])


#### Exercise 04:  Consider the target labels as **Survived Column** and remaining as the features 

* Print the shape of the features and labels


In [121]:
# YOUR CODE HERE
features = df.iloc[:,:-1]
labels = df.iloc[:,-1]
print(features.shape)
print(labels.shape)

(891, 11)
(891,)


#### Exercise 05:  Split the data into train and test sets




In [122]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state = 42)

In [128]:
df.fillna(0)

,PassengerId,Ticket_Class,Name,Sex,Age,Siblings_Spouse,Parents_Children,Ticket_Number,Fare,Cabin,Embarked,Survived
0,1,2,108,1,30.626179,1,0,523,7.2500,0,2,0
1,2,0,190,0,28.343690,1,0,596,71.2833,C85,0,1
2,3,2,353,0,28.343690,0,0,669,7.9250,0,2,1
3,4,0,272,0,28.343690,1,0,49,53.1000,C123,2,1
4,5,2,15,1,30.626179,0,0,472,8.0500,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,1,548,1,30.626179,0,0,101,13.0000,0,2,0
887,888,0,303,0,28.343690,0,0,14,30.0000,B42,2,1
888,889,2,413,0,30.626179,1,2,675,23.4500,0,2,0
889,890,0,81,1,28.343690,0,0,8,30.0000,C148,0,1


In [167]:
del df['Name']
df.head()

,PassengerId,Ticket_Class,Sex,Age,Siblings_Spouse,Parents_Children,Ticket_Number,Fare,Embarked,Survived
0,1,3rd class,male,22.0,1,0,A/5 21171,7.2500,Southampton,No
1,2,1st class,female,38.0,1,0,PC 17599,71.2833,Cherbourg,Yes
2,3,3rd class,female,26.0,0,0,STON/O2. 3101282,7.9250,Southampton,Yes
3,4,1st class,female,35.0,1,0,113803,53.1000,Southampton,Yes
4,5,3rd class,male,35.0,0,0,373450,8.0500,Southampton,No


In [185]:
df.head()
x=df.Ticket_Number[:]=0
print(x)

0


In [187]:
# YOUR CODE HERE
features = df.iloc[:,:-1]
labels = df.iloc[:,-1]
print(features.shape)
print(labels.shape)

(891, 9)
(891,)


#### Exercise 06: Build the classification model using bagging technique

In [188]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state = 42)
Bag = BaggingClassifier()

# YOUR CODE HERE to fit and predict
Bag.fit(X_train, y_train)

ValueError: ignored

'2020-09-26_13-06-15'

In [135]:
# YOUR CODE HERE to find the Accuracy of the Bagging Classifier model
y_predict = Bag.predict(X_test)

from sklearn import metrics

print(Bag.classes_)

score = metrics.accuracy_score(y_test,y_predict)
print(score)

[0 1]
0.5865921787709497


## Please answer the questions below to complete the experiment:

In [136]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good, But Not Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook  { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")